In [8]:
%cd practicas/

[Errno 2] No such file or directory: 'practicas/'
/workspace/practicas


/usr/local/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [9]:
from pyspark.sql import SparkSession

spark = ( SparkSession.builder
         .appName("pr503")
         .master("spark://spark-master:7077")
         .getOrCreate()
         )
 
sc = spark.sparkContext

In [23]:
from pyspark.sql.types import StructType, StructField, BooleanType, IntegerType, StringType, DoubleType, LongType, TimestampType
from pyspark.sql.functions import col, lit, lpad, upper, split, concat, concat_ws , substring, ceil, log10 ,least, regexp_replace, to_date, date_add, date_diff

schema_world = StructType([
    StructField("Place_Name", StringType(), True),
    StructField("Country", StringType(), True),
    StructField("City", StringType(), True),
    StructField("Annual_Visitors_Millions", DoubleType(), True),
    StructField("Type", StringType(), True),
    StructField("UNESCO_World_Heritage", StringType(), True),
    StructField("Year_Built", StringType(), True),
    StructField("Entry_Fee_USD", IntegerType(), True),
    StructField("Best_Visit_Month", StringType(), True),
    StructField("Region", StringType(), True),
    StructField("Tourism_Revenue_Million_USD", StringType(), True),
    StructField("Average_Visit_Duration_Hours", LongType(), True),
    StructField("Famous_For", StringType(), True),
])

df_city = (spark.read
             .format("csv")
             .schema(schema_world)
             .option("header", "True")
             .load("./data/world_famous_places_2024.csv"))
df_city.printSchema()
df_city.show(5)

root
 |-- Place_Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Annual_Visitors_Millions: double (nullable = true)
 |-- Type: string (nullable = true)
 |-- UNESCO_World_Heritage: string (nullable = true)
 |-- Year_Built: string (nullable = true)
 |-- Entry_Fee_USD: integer (nullable = true)
 |-- Best_Visit_Month: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Tourism_Revenue_Million_USD: string (nullable = true)
 |-- Average_Visit_Duration_Hours: long (nullable = true)
 |-- Famous_For: string (nullable = true)

+-------------------+-------------+----------------+------------------------+------------------+---------------------+----------------+-------------+-----------------+--------------+---------------------------+----------------------------+--------------------+
|         Place_Name|      Country|            City|Annual_Visitors_Millions|              Type|UNESCO_World_Heritage|      Year_Built|Ent

# Ejercicio 1: Generacion de codigos SKUs

In [11]:
df_feat = df_city.withColumn(
    "SKU_Lugar", 
    concat_ws(
        "_",
        substring(upper(col("Country")),1, 3),
        lpad(substring(upper(col("City")),1, 3), 3, "X"),
        col("Type")
    )
)
df_feat.show(5)

+-------------------+-------------+----------------+------------------------+------------------+---------------------+----------------+-------------+-----------------+--------------+---------------------------+----------------------------+--------------------+--------------------+
|         Place_Name|      Country|            City|Annual_Visitors_Millions|              Type|UNESCO_World_Heritage|      Year_Built|Entry_Fee_USD| Best_Visit_Month|        Region|Tourism_Revenue_Million_USD|Average_Visit_Duration_Hours|          Famous_For|           SKU_Lugar|
+-------------------+-------------+----------------+------------------------+------------------+---------------------+----------------+-------------+-----------------+--------------+---------------------------+----------------------------+--------------------+--------------------+
|       Eiffel Tower|       France|           Paris|                     7.0|    Monument/Tower|                   No|            1889|           35|May-J

# Ejercicio 2: Ajuste de precios y tiempos

In [13]:
df_feat = df_feat.withColumn(
    "Duracion_Techo",
    ceil(col("Average_Visit_Duration_Hours"))
)
df_feat = df_feat.withColumn(
    "Log_Ingresos",
    log10(col("Tourism_Revenue_Million_USD"))
)
df_feat = df_feat.withColumn(
    "Mejor_Oferta",
    least(col("Entry_Fee_USD"), lit(20))
)
df_feat.show(5)

+-------------------+-------------+----------------+------------------------+------------------+---------------------+----------------+-------------+-----------------+--------------+---------------------------+----------------------------+--------------------+--------------------+--------------+------------------+------------+
|         Place_Name|      Country|            City|Annual_Visitors_Millions|              Type|UNESCO_World_Heritage|      Year_Built|Entry_Fee_USD| Best_Visit_Month|        Region|Tourism_Revenue_Million_USD|Average_Visit_Duration_Hours|          Famous_For|           SKU_Lugar|Duracion_Techo|      Log_Ingresos|Mejor_Oferta|
+-------------------+-------------+----------------+------------------------+------------------+---------------------+----------------+-------------+-----------------+--------------+---------------------------+----------------------------+--------------------+--------------------+--------------+------------------+------------+
|       Eiffe

# Ejercicio 3: Limpieza de texto

In [ ]:
df_feat = df_feat.withColumn(
    "Desc_Corta",
    substring(col("Famous_For"), 1 , 15)
)

df_feat = df_feat.withColumn(
    "Ciudad_Limpia",
    regexp_replace(col("City"), "[a-z ]", "")
)
df_feat.show(5)


+-------------------+-------------+----------------+------------------------+------------------+---------------------+----------------+-------------+-----------------+--------------+---------------------------+----------------------------+--------------------+--------------------+--------------+------------------+------------+---------------+-------------+
|         Place_Name|      Country|            City|Annual_Visitors_Millions|              Type|UNESCO_World_Heritage|      Year_Built|Entry_Fee_USD| Best_Visit_Month|        Region|Tourism_Revenue_Million_USD|Average_Visit_Duration_Hours|          Famous_For|           SKU_Lugar|Duracion_Techo|      Log_Ingresos|Mejor_Oferta|     Desc_Corta|Ciudad_Limpia|
+-------------------+-------------+----------------+------------------------+------------------+---------------------+----------------+-------------+-----------------+--------------+---------------------------+----------------------------+--------------------+--------------------+-

# Ejercicio 4: Getsion de fechas de campaña

In [24]:
df_feat = df_feat.withColumn(
    "Inicio_Campana",
    to_date(lit("2024-06-01"))
)
df_feat = df_feat.withColumn(
    "Fin_Campana",
    date_add(col("Inicio_Campana"), 90)
)
df_feat = df_feat.withColumn(
    "Dias_Hasta_Fin",
    date_diff(
        col("Fin_Campana"),
        to_date(concat(col("Year_Built"), lit("-01-01")))
    )
)

df_feat.show(5)

+-------------------+-------------+----------------+------------------------+------------------+---------------------+----------------+-------------+-----------------+--------------+---------------------------+----------------------------+--------------------+--------------------+--------------+------------------+------------+---------------+-------------+--------------+-----------+--------------+
|         Place_Name|      Country|            City|Annual_Visitors_Millions|              Type|UNESCO_World_Heritage|      Year_Built|Entry_Fee_USD| Best_Visit_Month|        Region|Tourism_Revenue_Million_USD|Average_Visit_Duration_Hours|          Famous_For|           SKU_Lugar|Duracion_Techo|      Log_Ingresos|Mejor_Oferta|     Desc_Corta|Ciudad_Limpia|Inicio_Campana|Fin_Campana|Dias_Hasta_Fin|
+-------------------+-------------+----------------+------------------------+------------------+---------------------+----------------+-------------+-----------------+--------------+----------------